In [ ]:
import os
import cv2
import sys
import json
import shutil
import argparse
import subprocess
import numpy as np
import random
import matplotlib.pyplot as plt
from utils import read_gt_file, code_mask_to_labels, code_labels_to_colors, resize_image
%matplotlib inline

In [ ]:
colors = [[random.randint(0, 255) for _ in range(3)] for _ in range(80)] 
classes = { "ship": 0, "other": 1, "person": 2, "negative": 3 }
DATASET_PATH = '/data01/HiNAS-DATA/EO-DATA/RAW_DATA/mods/'

### Interact로 Masked Image 보기

In [ ]:
gt = read_gt_file(os.path.join(DATASET_PATH, 'mods.json'))
num_sequences = gt["dataset"]["num_seq"]

from ipywidgets import interact

@interact(seq_id=(1, num_sequences))
def show_sample(seq_id=0):
    fr_id = 0
    
    seq_path = gt['dataset']['sequences'][seq_id - 1]['path'][1:]
    seq_path_split = seq_path.split('/')
    
    # Load image
    img_name = gt['dataset']['sequences'][seq_id - 1]['frames'][fr_id]['image_file_name']
    img_name_split = img_name.split('.')
    
    img = cv2.imread(os.path.join(DATASET_PATH, "sequences", seq_path, img_name))
            
    if img is None:
        print('img is none')
        return
    
    # Convert BGR TO RGB for visualization
    # img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Visualize image
    obstacles = gt['dataset']['sequences'][seq_id - 1]['frames'][fr_id]['obstacles']
    
    # set rect_th for boxes
    rect_th = max(round(sum(img.shape) / 2 * 0.001), 1)
    # set text_th for category names
    text_th = max(rect_th - 1, 1)
    # set text_size for category names
    text_size = rect_th / 3
    
    for obj in obstacles:
        bbox = obj['bbox']
        name = obj['type']
        # BBOX given in Y_TL, X_TL, Y_BR, X_BR
        x_min = bbox[0]
        y_min = bbox[1]
        x_max = bbox[2]
        y_max = bbox[3]
        
        cv2.rectangle(img, 
                (x_min, y_min),
                (x_max, y_max), 
                color=colors[classes[name]],
                thickness=rect_th)
        
        p1, p2 = (int(x_min), int(y_min)), (int(x_max), int(y_max))
        label = f"{name}"
        w, h = cv2.getTextSize(label, 0, fontScale=text_size, thickness=text_th)[0]  # label width, height
        outside = p1[1] - h - 3 >= 0  # label fits outside box
        p2 = p1[0] + w, p1[1] - h - 3 if outside else p1[1] + h + 3
        
        cv2.rectangle(img, p1, p2, colors[classes[name]], -1, cv2.LINE_AA)  # filled
        cv2.putText(
            img,
            label,
            (p1[0], p1[1] - 2 if outside else p1[1] + h + 2),
            0,
            text_size,
            (255, 255, 255),
            thickness=text_th,
        )
        
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
    plt.figure(figsize=(16,8))
    plt.imshow(img)
    plt.axis('off')
    plt.show()

### Masked 이미지 저장

In [ ]:
def main():
    export_video = False
    # Create output folder for video export if it does not exist yet
    if export_video:
        pass
        # os.makedirs(os.path.join(args.output_path, args.method), exist_ok=True)

    # Read GT json
    gt = read_gt_file(os.path.join(DATASET_PATH, 'mods.json'))

    num_sequences = gt["dataset"]["num_seq"]

    for seq_id in range(num_sequences):
        # Create folders if they dont exist yet
        os.makedirs(os.path.join("results", 'seq%02d' % seq_id, 'tmp_frames'), exist_ok=True)

        num_frames_in_sequence = len(gt['dataset']['sequences'][seq_id - 1]['frames'])
        
        frame_counter = 0
        for fr_id in range(num_frames_in_sequence):
            sys.stdout.write("\rProcessing sequence %02d, image %03d / %03d\n" %
                                (seq_id, (fr_id + 1), num_frames_in_sequence))
            
            # feed, so it erases the previous line.
            sys.stdout.flush()

            seq_path = gt['dataset']['sequences'][seq_id - 1]['path'][1:]
            seq_path_split = seq_path.split('/')
            
            # Load image
            img_name = gt['dataset']['sequences'][seq_id - 1]['frames'][fr_id]['image_file_name']
            img_name_split = img_name.split('.')

            img = cv2.imread(os.path.join(DATASET_PATH, "sequences", seq_path, img_name))
            
            if img is None:
                print('img is none')
                break
            
            # Convert BGR TO RGB for visualization
            # img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

            # Visualize image
            obstacles = gt['dataset']['sequences'][seq_id - 1]['frames'][fr_id]['obstacles']
            
            # set rect_th for boxes
            rect_th = max(round(sum(img.shape) / 2 * 0.001), 1)
            # set text_th for category names
            text_th = max(rect_th - 1, 1)
            # set text_size for category names
            text_size = rect_th / 3
            
            for obj in obstacles:
                bbox = obj['bbox']
                name = obj['type']
                # BBOX given in Y_TL, X_TL, Y_BR, X_BR
                x_min = bbox[0]
                y_min = bbox[1]
                x_max = bbox[2]
                y_max = bbox[3]
                
                cv2.rectangle(img, 
                      (x_min, y_min),
                      (x_max, y_max), 
                      color=colors[classes[name]],
                      thickness=rect_th)
                
                p1, p2 = (int(x_min), int(y_min)), (int(x_max), int(y_max))
                label = f"{name}"
                w, h = cv2.getTextSize(label, 0, fontScale=text_size, thickness=text_th)[0]  # label width, height
                outside = p1[1] - h - 3 >= 0  # label fits outside box
                p2 = p1[0] + w, p1[1] - h - 3 if outside else p1[1] + h + 3
                
                cv2.rectangle(img, p1, p2, colors[classes[name]], -1, cv2.LINE_AA)  # filled
                cv2.putText(
                    img,
                    label,
                    (p1[0], p1[1] - 2 if outside else p1[1] + h + 2),
                    0,
                    text_size,
                    (255, 255, 255),
                    thickness=text_th,
                )
                
            cv2.imwrite(os.path.join("results", 'seq%02d' % seq_id,
                                            'tmp_frames', '%08d.png' % fr_id), img)
            frame_counter += 1

if __name__ == "__main__":
    main()